In [13]:
# lambdas
# sol-hnt: 0.25
# msol-mnde: 6.31
# sol-usdc: 0.014
# sol-pengu: 0.48

In [14]:
import pandas as pd
import numpy as np
import datetime
from pprint import pprint

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [15]:
df = pd.read_csv("/content/drive/MyDrive/colosseum_hackathon/datasets/sol_pengu_pool/sol_pengu_pool_swaps.csv")

In [16]:
df.head()

,slot,block_time,tx_signature,token_mint_a,token_mint_b,token_vault_a,token_vault_b,num_swaps,token_amount_a,token_amount_b,pre_balance_a,pre_balance_b,post_balance_a,post_balance_b,decimals_a,decimals_b,token_price_a,token_price_b,token_ema_a,token_ema_b
0,373348755,1760458842,4uPbbFd37Rqyy8QFSw6KivndYWQteLMSAUabKcdAuYHNCD...,So11111111111111111111111111111111111111112,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,J757hq9DXGPDYfCoeGpTcD9A71NFgNqBRMXHrdVGyRxK,SdFLxX6sWTkKWje3Xb4YNewbm5ieaj3tfEJYeLTyqyg,1,2795627,22913485,14387.629156,1.902471e+08,14387.626360,1.902471e+08,9,6,204.702406,0.025035,198.292343,0.024316
1,373348586,1760458774,3r5TJs6otoo8enW1VgkXULf2UzdDsoKYtqVd8LQ4mtxBPr...,So11111111111111111111111111111111111111112,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,J757hq9DXGPDYfCoeGpTcD9A71NFgNqBRMXHrdVGyRxK,SdFLxX6sWTkKWje3Xb4YNewbm5ieaj3tfEJYeLTyqyg,1,3010327305,24669982850,14390.639483,1.902224e+08,14387.629156,1.902471e+08,9,6,202.933575,0.024761,198.224513,0.024307
2,373348586,1760458774,aAMzbNpGKempL8zZFpMmHeaRtjxtXHtV2nxE2f6j294yhK...,So11111111111111111111111111111111111111112,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,J757hq9DXGPDYfCoeGpTcD9A71NFgNqBRMXHrdVGyRxK,SdFLxX6sWTkKWje3Xb4YNewbm5ieaj3tfEJYeLTyqyg,1,1124893147,9216977980,14391.764376,1.902132e+08,14390.639483,1.902224e+08,9,6,202.933575,0.024761,198.224513,0.024307
3,373348546,1760458758,5nHYm87ZnisgxQdWpHcHWikWbLjy7TnD1LhDaBLuuwPm7Z...,So11111111111111111111111111111111111111112,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,J757hq9DXGPDYfCoeGpTcD9A71NFgNqBRMXHrdVGyRxK,SdFLxX6sWTkKWje3Xb4YNewbm5ieaj3tfEJYeLTyqyg,1,1093463768,8958595270,14392.857840,1.902043e+08,14391.764376,1.902132e+08,9,6,202.651126,0.024727,198.207664,0.024305
4,373348543,1760458757,5UdTruM4LhsDHEUdJWtS8Bed9LRrGwac1yMnAmAFYbjHrd...,So11111111111111111111111111111111111111112,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,J757hq9DXGPDYfCoeGpTcD9A71NFgNqBRMXHrdVGyRxK,SdFLxX6sWTkKWje3Xb4YNewbm5ieaj3tfEJYeLTyqyg,1,1124018075,9208037376,14393.981858,1.901950e+08,14392.857840,1.902043e+08,9,6,202.650163,0.024728,198.206866,0.024304


In [17]:
df['pool_address'] = "CSP4RmB6kBHkKGkyTnzt9zYYXDA8SbZ5Do5WfZcjqjE4"
df['fee_tier'] = 0.0005
df['token_amount_a_ui'] = df['token_amount_a'] / 10 ** df['decimals_a']
df['token_amount_b_ui'] = df['token_amount_b'] / 10 ** df['decimals_b']
df['volume_usd'] = df['token_amount_a_ui'] * df['token_price_a']
df['fee_usd'] = df['volume_usd'] * df['fee_tier']
df['lp_fee_usd'] = df['fee_usd'] * 0.87
df['date'] = pd.to_datetime(df['block_time'], unit='s')
df['price_ratio'] = df['token_price_a'] / df['token_price_b']
df['tvl_usd'] = (df['post_balance_a'] * df['token_price_a'] + df['post_balance_b'] * df['token_price_b'])
df['tvl_utilization'] = df['volume_usd'] / df['tvl_usd']

df['balance_ratio'] = (df['post_balance_a'] * df['token_price_a']) / (df['post_balance_b'] * df['token_price_b'])
df['balance_imbalance'] = abs(df['balance_ratio'] - 1.0)

In [18]:
df = df.set_index('date')

In [19]:
df.head()

,slot,block_time,tx_signature,token_mint_a,token_mint_b,token_vault_a,token_vault_b,num_swaps,token_amount_a,token_amount_b,pre_balance_a,pre_balance_b,post_balance_a,post_balance_b,decimals_a,decimals_b,token_price_a,token_price_b,token_ema_a,token_ema_b,pool_address,fee_tier,token_amount_a_ui,token_amount_b_ui,volume_usd,fee_usd,lp_fee_usd,price_ratio,tvl_usd,tvl_utilization,balance_ratio,balance_imbalance
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-10-14 16:20:42,373348755,1760458842,4uPbbFd37Rqyy8QFSw6KivndYWQteLMSAUabKcdAuYHNCD...,So11111111111111111111111111111111111111112,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,J757hq9DXGPDYfCoeGpTcD9A71NFgNqBRMXHrdVGyRxK,SdFLxX6sWTkKWje3Xb4YNewbm5ieaj3tfEJYeLTyqyg,1,2795627,22913485,14387.629156,1.902471e+08,14387.626360,1.902471e+08,9,6,204.702406,0.025035,198.292343,0.024316,CSP4RmB6kBHkKGkyTnzt9zYYXDA8SbZ5Do5WfZcjqjE4,0.0005,0.002796,22.913485,0.572272,0.000286,0.000249,8176.802421,7.707929e+06,7.424453e-08,0.618379,0.381621
2025-10-14 16:19:34,373348586,1760458774,3r5TJs6otoo8enW1VgkXULf2UzdDsoKYtqVd8LQ4mtxBPr...,So11111111111111111111111111111111111111112,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,J757hq9DXGPDYfCoeGpTcD9A71NFgNqBRMXHrdVGyRxK,SdFLxX6sWTkKWje3Xb4YNewbm5ieaj3tfEJYeLTyqyg,1,3010327305,24669982850,14390.639483,1.902224e+08,14387.629156,1.902471e+08,9,6,202.933575,0.024761,198.224513,0.024307,CSP4RmB6kBHkKGkyTnzt9zYYXDA8SbZ5Do5WfZcjqjE4,0.0005,3.010327,24669.982850,610.896482,0.305448,0.265740,8195.597847,7.630497e+06,8.005986e-05,0.619800,0.380200
2025-10-14 16:19:34,373348586,1760458774,aAMzbNpGKempL8zZFpMmHeaRtjxtXHtV2nxE2f6j294yhK...,So11111111111111111111111111111111111111112,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,J757hq9DXGPDYfCoeGpTcD9A71NFgNqBRMXHrdVGyRxK,SdFLxX6sWTkKWje3Xb4YNewbm5ieaj3tfEJYeLTyqyg,1,1124893147,9216977980,14391.764376,1.902132e+08,14390.639483,1.902224e+08,9,6,202.933575,0.024761,198.224513,0.024307,CSP4RmB6kBHkKGkyTnzt9zYYXDA8SbZ5Do5WfZcjqjE4,0.0005,1.124893,9216.977980,228.278588,0.114139,0.099301,8195.597847,7.630497e+06,2.991661e-05,0.620010,0.379990
2025-10-14 16:19:18,373348546,1760458758,5nHYm87ZnisgxQdWpHcHWikWbLjy7TnD1LhDaBLuuwPm7Z...,So11111111111111111111111111111111111111112,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,J757hq9DXGPDYfCoeGpTcD9A71NFgNqBRMXHrdVGyRxK,SdFLxX6sWTkKWje3Xb4YNewbm5ieaj3tfEJYeLTyqyg,1,1093463768,8958595270,14392.857840,1.902043e+08,14391.764376,1.902132e+08,9,6,202.651126,0.024727,198.207664,0.024305,CSP4RmB6kBHkKGkyTnzt9zYYXDA8SbZ5Do5WfZcjqjE4,0.0005,1.093464,8958.595270,221.591664,0.110796,0.096392,8195.550291,7.619904e+06,2.908064e-05,0.620085,0.379915
2025-10-14 16:19:17,373348543,1760458757,5UdTruM4LhsDHEUdJWtS8Bed9LRrGwac1yMnAmAFYbjHrd...,So11111111111111111111111111111111111111112,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,J757hq9DXGPDYfCoeGpTcD9A71NFgNqBRMXHrdVGyRxK,SdFLxX6sWTkKWje3Xb4YNewbm5ieaj3tfEJYeLTyqyg,1,1124018075,9208037376,14393.981858,1.901950e+08,14392.857840,1.902043e+08,9,6,202.650163,0.024728,198.206866,0.024304,CSP4RmB6kBHkKGkyTnzt9zYYXDA8SbZ5Do5WfZcjqjE4,0.0005,1.124018,9208.037376,227.782446,0.113891,0.099085,8195.133508,7.620107e+06,2.989229e-05,0.620130,0.379870


In [20]:
# aggregation analysis
swap_rate = df['num_swaps'].resample('1min').sum().mean()

In [21]:
-np.log(0.3) / swap_rate

np.float64(0.4882611317134597)

In [ ]:
# after aggregate
df['change_token_a'] = df['token_price_a'].pct_change()
df['change_token_b'] = df['token_price_b'].pct_change()

df['price_ema_deviation_a'] = (df['token_price_a'] - df['token_ema_a']) / df['token_ema_a']
df['price_ema_deviation_b'] = (df['token_price_b'] - df['token_ema_b']) / df['token_ema_b']

window = 3
df['vol_token_a'] = df['change_token_a'].rolling(window).std()
df['vol_token_b'] = df['change_token_b'].rolling(window).std()
df['vol_ratio'] = df['volatility_a'] / (df['volatility_b'] + 1e-8)

df['price_ratio_change'] = df['price_ratio'].pct_change()

df['utilization_a'] = abs(df['token_amount_a']) / df['pre_balance_a']
df['utilization_b'] = abs(df['token_amount_b']) / df['pre_balance_b']

df['tvl_change'] = df['tvl_usd'].pct_change()

df['volume_per_swap'] = df['volume_usd'] / df['num_swaps']

df['time_diff_minutes'] = df['block_time'].diff().dt.total_seconds() / 60
df['swap_rate'] = df['num_swaps'] / df['time_diff_minutes'].fillna(5)

# ===== 4. IMPERMANENT LOSS (3 features) =====
print("  ✓ Impermanent loss features")
initial_price_ratio = df['price_ratio'].iloc[0] if len(df) > 0 else 1.0
df['price_ratio_vs_initial'] = df['price_ratio'] / initial_price_ratio
# IL formula: 2*sqrt(r) / (1 + r) - 1
df['impermanent_loss'] = (2 * np.sqrt(df['price_ratio_vs_initial']) /
                          (1 + df['price_ratio_vs_initial']) - 1)
df['il_change'] = df['impermanent_loss'].diff()


df['avg_swap_size_a'] = df['token_amount_a'] / df['num_swaps']
df['avg_swap_size_b'] = df['token_amount_b'] / df['num_swaps']

df['fee_revenue_per_swap'] = df['estimated_fee_revenue'] / df['num_swaps']
df['fee_efficiency'] = df['estimated_fee_revenue'] / (df['volume_usd'] + 1e-8)
df['revenue_per_liquidity'] = df['estimated_fee_revenue'] / df['tvl_usd']

In [12]:
df.head()

,slot,block_time,tx_signature,token_mint_a,token_mint_b,token_vault_a,token_vault_b,num_swaps,token_amount_a,token_amount_b,pre_balance_a,pre_balance_b,post_balance_a,post_balance_b,decimals_a,decimals_b,token_price_a,token_price_b,token_ema_a,token_ema_b,pool_address,fee_tier,token_amount_a_ui,token_amount_b_ui,volume_usd,fee_usd,lp_fee_usd,date
0,372117422,1759970010,voFRsYQztPxrn8Up585cw57YZ1FynubwpicBzc7Sy8gJnL...,So11111111111111111111111111111111111111112,hntyVP6YFm1Hg25TN9WGLqM12b8TQmcknKrdu1oxWux,BW18gGtaQjD2VgNmmWXBeqHu22k4KC4WamWyEP3S7PNU,BvquGcdP4bVHfb6RxBxUSaS2kwm36FPHMadACfYKMp6t,1,615874003,5636897771,264.814110,73738.018574,265.429984,73681.649597,9,8,228.308546,2.497639,228.811711,2.499824,CSP4RmB6kBHkKGkyTnzt9zYYXDA8SbZ5Do5WfZcjqjE4,0.0005,0.615874,56.368978,140.609298,0.070305,0.061165,2025-10-09 00:33:30
1,372117410,1759970005,3D99LotRruAu3DKzXy8zv4tu28v5dwpAdFagx2wzQT3j8L...,So11111111111111111111111111111111111111112,hntyVP6YFm1Hg25TN9WGLqM12b8TQmcknKrdu1oxWux,BW18gGtaQjD2VgNmmWXBeqHu22k4KC4WamWyEP3S7PNU,BvquGcdP4bVHfb6RxBxUSaS2kwm36FPHMadACfYKMp6t,1,59736975,546843607,264.754373,73743.487011,264.814110,73738.018574,9,8,228.332011,2.497721,228.812321,2.499827,CSP4RmB6kBHkKGkyTnzt9zYYXDA8SbZ5Do5WfZcjqjE4,0.0005,0.059737,5.468436,13.639864,0.006820,0.005933,2025-10-09 00:33:25
2,372117210,1759969926,4jD9zFY8wTNkE9Xw6YVATdcyTfr9Pq7UPZn81LzNp7wqCp...,So11111111111111111111111111111111111111112,hntyVP6YFm1Hg25TN9WGLqM12b8TQmcknKrdu1oxWux,BW18gGtaQjD2VgNmmWXBeqHu22k4KC4WamWyEP3S7PNU,BvquGcdP4bVHfb6RxBxUSaS2kwm36FPHMadACfYKMp6t,1,1409922915,12911334671,263.344450,73872.600357,264.754373,73743.487011,9,8,228.985100,2.498548,228.814718,2.499862,CSP4RmB6kBHkKGkyTnzt9zYYXDA8SbZ5Do5WfZcjqjE4,0.0005,1.409923,129.113347,322.851340,0.161426,0.140440,2025-10-09 00:32:06
3,372117112,1759969888,2KoUoEE6eNJbLKNbzB6SSWV9MDTDuXyiNLi3F9KWwY4ULk...,So11111111111111111111111111111111111111112,hntyVP6YFm1Hg25TN9WGLqM12b8TQmcknKrdu1oxWux,BW18gGtaQjD2VgNmmWXBeqHu22k4KC4WamWyEP3S7PNU,BvquGcdP4bVHfb6RxBxUSaS2kwm36FPHMadACfYKMp6t,1,231058967,2118644068,263.575509,73851.413917,263.344450,73872.600357,9,8,229.063501,2.499054,228.812174,2.499868,CSP4RmB6kBHkKGkyTnzt9zYYXDA8SbZ5Do5WfZcjqjE4,0.0005,0.231059,21.186441,52.927176,0.026464,0.023023,2025-10-09 00:31:28
4,372117071,1759969872,2H5s9zUcuSc93Akphew19gTZc8UpSjC8RzoYLRfofxMCto...,So11111111111111111111111111111111111111112,hntyVP6YFm1Hg25TN9WGLqM12b8TQmcknKrdu1oxWux,BW18gGtaQjD2VgNmmWXBeqHu22k4KC4WamWyEP3S7PNU,BvquGcdP4bVHfb6RxBxUSaS2kwm36FPHMadACfYKMp6t,1,22885893,209834120,263.598395,73849.315575,263.575509,73851.413917,9,8,228.977706,2.498333,228.811211,2.499872,CSP4RmB6kBHkKGkyTnzt9zYYXDA8SbZ5Do5WfZcjqjE4,0.0005,0.022886,2.098341,5.240359,0.002620,0.002280,2025-10-09 00:31:12
